In [1]:
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [3]:
data_text_file='/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/data_tissues.txt'
Image.MAX_IMAGE_PIXELS = None
with open(data_text_file, 'r') as file:
    lines = file.readlines()
test_files=lines[0:50]
train_files=lines[50:]

len(test_files), len(train_files)   

(50, 11)

In [42]:
for line in test_files:
    print(line)
    if 'TISSUE' in line:
        sub_tissue_name=(line.split('/')[4].split('_')[0])+'_'+(line.split('/')[4].split('_')[1])
        sub_name=(line.split('/')[4].split('_')[0])
    else:
        sub_name=(line.split('/')[4].split('_')[0])
        sub_tissue_name=sub_name
    set_name=line.split('/')[1]
    
    set_dir=f'/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/{set_name}/{sub_name}/Registered'
    file_names = [name for name in os.listdir(set_dir) if sub_tissue_name in name and 'DAPI' in name]
    sorted_file_names = sorted(file_names, key=lambda name: int(name.split('_ROUND_')[1].split('.')[0]))
    target_file=f'{set_dir}/{sorted_file_names[0]}'
    source_files_1=f'{set_dir}/{sorted_file_names[10]}'
    source_files_2=f'{set_dir}/{sorted_file_names[15]}'
    source_files_3=f'{set_dir}/{sorted_file_names[-1]}'
    mask=f'/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/{set_name}/{sub_name}/Registered/{sub_tissue_name}_RetentionMask.tif'
    target = np.array(Image.open(target_file))/255.
    source_1 = np.array(Image.open(source_files_1))/255.
    source_2 = np.array(Image.open(source_files_2))/255.
    source_3 = np.array(Image.open(source_files_3))/255.
    mask = np.array(Image.open(mask))
    target = target * mask
    source_1 = source_1 * mask
    source_2 = source_2 * mask
    source_3 = source_3 * mask
    window_size = (1024, 1024)

    # Define directories
    output_dir = '/home-local/rudravg/test_DAPI/1024_Dataset'
    target_dir = os.path.join(output_dir, 'target_images')
    source_dir = os.path.join(output_dir, 'source_images')

    # Create directories if they don't exist
    os.makedirs(target_dir, exist_ok=True)
    os.makedirs(source_dir, exist_ok=True)

    # Define pairs file
    pairs_file = os.path.join(output_dir, 'pairs.txt')

    # Open pairs file for writing
    with open(pairs_file, 'a') as f:
        for i in range(0, mask.shape[0], window_size[0]):
            for j in range(0, mask.shape[1], window_size[1]):
                window_mask = mask[i:i+window_size[0], j:j+window_size[1]]
                
                if np.count_nonzero(window_mask) > 400:
                    # Save the cut pieces of the target in target_dir
                    window_target = target[i:i+window_size[0], j:j+window_size[1]]
                    target_path = os.path.join(target_dir, f'{sub_name}_{set_name}_Cut_{i}_{j}.npy')
                    #target_png_path = os.path.join(target_dir, f'{sub_name}_{set_name}_{i}_{j}.png')
                    np.save(target_path, window_target)
                   # Image.fromarray(window_target).save(target_png_path)

                    
                    # Cut source_1, source_2 and source_3 in the same way and save their corresponding pieces in source_dir
                    for source, source_name in zip([source_1, source_2, source_3], ['source_1', 'source_2', 'source_3']):
                        window_source = source[i:i+window_size[0], j:j+window_size[1]]
                        source_path = os.path.join(source_dir, f'Sub_{sub_name}_Set_{set_name}_Round_{source_name}_Cut_{i}_{j}.npy')
                       # source_png_path = os.path.join(source_dir, f'{sub_name}_{set_name}_{source_name}_{i}_{j}.png')
                       # Image.fromarray(window_source).save(source_png_path)
                        np.save(source_path, window_source)
                        
                        # Append a space separated line in pairs_file with the first part being path to the source image and the second part being path to the target image
                        f.write(f'{source_path} {target_path}\n')

./Set03/GCA077TIA/Registered/GCA077TIA_DAPI_DAPI_12ms_ROUND_00.tif

./Set03/GCA019ACA/Registered/GCA019ACA_DAPI_DAPI_12ms_ROUND_00.tif

./Set03/GCA071TIA/Registered/GCA071TIA_DAPI_DAPI_12ms_ROUND_00.tif

./Set02/GCA201ACB/Registered/GCA201ACB_DAPI_DAPI_12ms_ROUND_00.tif



: 

In [39]:
tial=np.load('/home-local/rudravg/test_DAPI/1024_Dataset/source_images/GCA077TIA_Set03_source_1_0_1024.npy')

In [40]:
np.max(tial)

1.0

In [41]:
np.unique(tial)

array([0.        , 0.00784314, 0.01176471, 0.01568627, 0.01960784,
       0.02352941, 0.02745098, 0.03137255, 0.03529412, 0.03921569,
       0.04313725, 0.04705882, 0.05098039, 0.05490196, 0.05882353,
       0.0627451 , 0.06666667, 0.07058824, 0.0745098 , 0.07843137,
       0.08235294, 0.08627451, 0.09019608, 0.09411765, 0.09803922,
       0.10196078, 0.10588235, 0.10980392, 0.11372549, 0.11764706,
       0.12156863, 0.1254902 , 0.12941176, 0.13333333, 0.1372549 ,
       0.14117647, 0.14509804, 0.14901961, 0.15294118, 0.15686275,
       0.16078431, 0.16470588, 0.16862745, 0.17254902, 0.17647059,
       0.18039216, 0.18431373, 0.18823529, 0.19215686, 0.19607843,
       0.2       , 0.20392157, 0.20784314, 0.21176471, 0.21568627,
       0.21960784, 0.22352941, 0.22745098, 0.23137255, 0.23529412,
       0.23921569, 0.24313725, 0.24705882, 0.25098039, 0.25490196,
       0.25882353, 0.2627451 , 0.26666667, 0.27058824, 0.2745098 ,
       0.27843137, 0.28235294, 0.28627451, 0.29019608, 0.29411

In [27]:
np.max(mask),np.max(target),np.max(source_1),np.max(source_2),np.max(source_3)

(1.0, 1.0, 1.0, 1.0, 1.0)

In [28]:
np.min(mask),np.min(target),np.min(source_1),np.min(source_2),np.min(source_3)

(0.0, 0.0, 0.0, 0.0, 0.0)

In [29]:
np.unique(target)

array([0.        , 0.00392157, 0.00784314, 0.01568627, 0.01960784,
       0.02352941, 0.02745098, 0.03137255, 0.03529412, 0.03921569,
       0.04313725, 0.04705882, 0.05098039, 0.05490196, 0.05882353,
       0.0627451 , 0.06666667, 0.07058824, 0.0745098 , 0.07843137,
       0.08235294, 0.08627451, 0.09019608, 0.09411765, 0.09803922,
       0.10196078, 0.10588235, 0.10980392, 0.11372549, 0.11764706,
       0.12156863, 0.1254902 , 0.12941176, 0.13333333, 0.1372549 ,
       0.14117647, 0.14509804, 0.14901961, 0.15294118, 0.15686275,
       0.16078431, 0.16470588, 0.16862745, 0.17254902, 0.17647059,
       0.18039216, 0.18431373, 0.18823529, 0.19215686, 0.19607843,
       0.2       , 0.20392157, 0.20784314, 0.21176471, 0.21568627,
       0.21960784, 0.22352941, 0.22745098, 0.23137255, 0.23529412,
       0.23921569, 0.24313725, 0.24705882, 0.25098039, 0.25490196,
       0.25882353, 0.2627451 , 0.26666667, 0.27058824, 0.2745098 ,
       0.27843137, 0.28235294, 0.28627451, 0.29019608, 0.29411

In [5]:
import os
import numpy as np
from tqdm import tqdm

# Define the directory
dir_path = '/home-local/rudravg/test_DAPI/1024_Dataset/target_images'
non_512_filenames = []
# Iterate over all files in the directory
for filename in tqdm(os.listdir(dir_path)):
    # Check if the file is a .npy file
    if filename.endswith('.npy'):
        # Load the numpy array
        array = np.load(os.path.join(dir_path, filename))
        
        # Check if the shape of the array is not 512x512
        if array.shape != (1024, 1024):
            print(f'Filename: {filename}, Shape: {array.shape}')
            #non_512_filenames.append(filename)
#print(len(non_512_filenames))

100%|██████████| 2792/2792 [00:09<00:00, 289.56it/s]
